In [ ]:
# Traditional Conjoint Analysis (Python)
# prepare for Python version 3x features and functions
from __future__ import division, print_function
# import packages for analysis and modeling


In [ ]:
import pandas as pd # data frame operations
import numpy as np # arrays and math functions
import statsmodels.api as sm # statistical models (including regression)
import statsmodels.formula.api as smf # R-like model specification
from patsy.contrasts import Sum
# read in conjoint survey profiles with respondent ranks
#conjoint_data_frame = pd.read_csv('mobile_services_ranking.csv')
from google.colab import files
uploaded = files.upload()


Saving mobile_services_ranking.csv to mobile_services_ranking.csv


In [ ]:
conjoint_data_frame = pd.read_csv('mobile_services_ranking.csv')
conjoint_data_frame.head()

,brand,startup,monthly,service,retail,apple,samsung,google,ranking
0,"""AT&T""","""$100""","""$100""","""4G NO""","""Retail NO""","""Apple NO""","""Samsung NO""","""Nexus NO""",11
1,"""Verizon""","""$300""","""$100""","""4G NO""","""Retail YES""","""Apple YES""","""Samsung YES""","""Nexus NO""",12
2,"""US Cellular""","""$400""","""$200""","""4G NO""","""Retail NO""","""Apple NO""","""Samsung YES""","""Nexus NO""",9
3,"""Verizon""","""$400""","""$400""","""4G YES""","""Retail YES""","""Apple NO""","""Samsung NO""","""Nexus NO""",2
4,"""Verizon""","""$200""","""$300""","""4G NO""","""Retail NO""","""Apple NO""","""Samsung YES""","""Nexus YES""",8


In [ ]:
# set up sum contrasts for effects coding as needed for conjoint analysis
# using C(effect, Sum) notation within main effects model specification
main_effects_model = 'ranking ~ C(brand, Sum) + C(startup, Sum) + \
C(monthly, Sum) + C(service, Sum) + C(retail, Sum) + C(apple, Sum) + \
C(samsung, Sum) + C(google, Sum)'
main_effects_model

'ranking ~ C(brand, Sum) + C(startup, Sum) + C(monthly, Sum) + C(service, Sum) + C(retail, Sum) + C(apple, Sum) + C(samsung, Sum) + C(google, Sum)'

In [ ]:
# fit linear regression model using main effects only (no interaction terms)
main_effects_model_fit = \
smf.ols(main_effects_model, data = conjoint_data_frame).fit()




In [ ]:
print(main_effects_model_fit.summary())



                            OLS Regression Results                            
Dep. Variable:                ranking   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     97.07
Date:                Tue, 16 Mar 2021   Prob (F-statistic):             0.0794
Time:                        13:35:27   Log-Likelihood:                 10.568
No. Observations:                  16   AIC:                             8.864
Df Residuals:                       1   BIC:                             20.45
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


In [ ]:
conjoint_attributes = ['brand', 'startup', 'monthly', 'service', \
'retail', 'apple', 'samsung', 'google']
conjoint_attributes

['brand',
 'startup',
 'monthly',
 'service',
 'retail',
 'apple',
 'samsung',
 'google']

In [ ]:
# build part-worth information one attribute at a time
level_name = []
part_worth = []
part_worth_range = []
end = 1 # initialize index for coefficient in params

In [ ]:
list(conjoint_data_frame[item])

['"AT&T"',
 '"Verizon"',
 '"US Cellular"',
 '"Verizon"',
 '"Verizon"',
 '"Verizon"',
 '"US Cellular"',
 '"AT&T"',
 '"AT&T"',
 '"T-Mobile"',
 '"US Cellular"',
 '"T-Mobile"',
 '"T-Mobile"',
 '"US Cellular"',
 '"T-Mobile"',
 '"AT&T"']

In [ ]:
import numpy as np

for item in conjoint_attributes:
  nlevels = len(list(np.unique(conjoint_data_frame[item]))) #devuelve la longitud de una cadena de caracteres o el número de elementos de una lista
  level_name.append(list(np.unique(conjoint_data_frame[item])))
  begin = end
  end = begin + nlevels - 1
  new_part_worth = list(main_effects_model_fit.params[begin:end])
  new_part_worth.append((-1) * sum(new_part_worth))
  part_worth_range.append(max(new_part_worth) - min(new_part_worth))
  part_worth.append(new_part_worth)
# end set to begin next iteration

In [ ]:
# compute attribute relative importance values from ranges
attribute_importance = []

for item in part_worth_range:
  attribute_importance.append(round(100 * (item / sum(part_worth_range)),2))


In [ ]:
# user-defined dictionary for printing descriptive attribute names
effect_name_dict = {'brand' : 'Mobile Service Provider', \
'startup' : 'Start-up Cost', 'monthly' : 'Monthly Cost', \
'service' : 'Offers 4G Service', 'retail' : 'Has Nearby Retail Store', \
'apple' : 'Sells Apple Products', 'samsung' : 'Sells Samsung Products', \
'google' : 'Sells Google/Nexus Products'}


In [ ]:
# report conjoint measures to console
index = 0 # initialize for use in for-loop

for item in conjoint_attributes:
  print('\nAttribute:', effect_name_dict[item])
  print(' Importance:', attribute_importance[index])
  print(' Level Part-Worths')
  for level in range(len(level_name[index])):
    print(' ',level_name[index][level], part_worth[index][level]) 
  index = index + 1


Attribute: Mobile Service Provider
 Importance: 2.38
 Level Part-Worths
  "AT&T" -8.881784197001252e-16
  "T-Mobile" -0.24999999999999944
  "US Cellular" 2.220446049250313e-16
  "Verizon" 0.2500000000000001

Attribute: Start-up Cost
 Importance: 7.14
 Level Part-Worths
  "$100" 0.7500000000000089
  "$200" -1.3711254354120683e-14
  "$300" 3.1086244689504383e-15
  "$400" -0.7499999999999982

Attribute: Monthly Cost
 Importance: 51.19
 Level Part-Worths
  "$100" 5.000000000000005
  "$200" 2.000000000000001
  "$300" -1.2500000000000053
  "$400" -5.750000000000001

Attribute: Offers 4G Service
 Importance: 16.67
 Level Part-Worths
  "4G NO" -1.7500000000000007
  "4G YES" 1.7500000000000007

Attribute: Has Nearby Retail Store
 Importance: 2.38
 Level Part-Worths
  "Retail NO" 0.25000000000000067
  "Retail YES" -0.25000000000000067

Attribute: Sells Apple Products
 Importance: 2.38
 Level Part-Worths
  "Apple NO" 0.25000000000000044
  "Apple YES" -0.25000000000000044

Attribute: Sells Samsun